In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


In [196]:
assistant = client.beta.assistants.create(
    name='Math tutor',
    instructions='You are a personal math tutor. Write and run code to answer math questions.',
    tools=[{'type': 'code_interpreter'}],
    model='gpt-3.5-turbo',
)



In [197]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_7RrF07t4koQbmcrKj3xchxIK', created_at=1719972491, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [198]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'Solve this problem: 3x + 11 = 14'
)

In [199]:
print(message)

Message(id='msg_5VvQZGcmBOZklUPftbkHPu6N', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Solve this problem: 3x + 11 = 14'), type='text')], created_at=1719972491, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_7RrF07t4koQbmcrKj3xchxIK')


In [200]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    
    assistant_id= assistant.id,
)

In [201]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [202]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [203]:
for message in reversed(messages.data):
    print(message.role + ": " + message.content[0].text.value)

user: Solve this problem: 3x + 11 = 14


In [204]:
file = client.files.create(
    file=open("memgpt-paper.pdf", "rb"),
    purpose= "assistants"
)

In [205]:
print(file)

FileObject(id='file-zJzbAHvKY3zkrppkqWaPLPzm', bytes=663708, created_at=1719972492, filename='memgpt-paper.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [206]:
assistant = client.beta.assistants.create(
  name="Machine Learning Researcher",
  instructions="'You are a machine learning researcher, answer questions on the research paper.'",
  model="gpt-3.5-turbo",
  tools=[{"type": "file_search"}],
)

In [207]:
vector_store = client.beta.vector_stores.create(name='memgpt-paper')
file_paths = ['./memgpt-paper.pdf']
file_streams = [open(file_path, 'rb') for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams,
)
print(file_batch.status)
print(file_batch.file_counts)
print(file_batch)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
VectorStoreFileBatch(id='vsfb_51f2c21e03ba4ddba0ba64dfd32766d5', created_at=1719972495, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), object='vector_store.file_batch', status='completed', vector_store_id='vs_GYz9T0roZzFl0cNb8CN72rng')


In [208]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)



In [209]:
message_file = client.files.create(
    file=open("memgpt-paper.pdf", "rb"), purpose="assistants"
)

thread = client.beta.threads.create(
    messages=[
        {
            'role' : 'user',
            'content' : 'How does memgpt allow LLMs to have unlimited context length?',
            'attachments' : [
                {'file_id': message_file.id, 'tools': ['file_search'] },
            ],
        }
    ]
)
    
print(thread.tool_resources.file_search)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[0].attachments[0].tools[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages[0].attachments[0].tools[0]', 'code': 'invalid_type'}}

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'How does memgpt allow LLMs to have unlimited context length?'
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= assistant.id,
)

In [ ]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [ ]:
for message in reversed(messages.data):
    print(message.role + ": " + message.content[0].text.value)

user: Solve this problem: 3x + 11 = 14
